<a href="https://colab.research.google.com/github/UFM-Market-Trends/UFM-Market-Trends-SDK/blob/main/web-scrapping/0_Webscrapping_Banguat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 3.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 21 not upgraded.
Need to get 95.6 MB of archives.
After this operation, 321 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 108.0.5359.71-0ubuntu0.18.04.5 [1,159 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 108.0.5359.71-0ubuntu0.18

In [3]:
from selenium.webdriver.common.by import By

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver

option = webdriver.ChromeOptions()
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--disable-dev-sh-usage')
driver = webdriver.Chrome("/usr/bin/chromedriver", # https://stackoverflow.com/questions/60686871/colab-cant-find-chromedriver-path
                          options=option)

# driver.get('https://www.imdb.com/chart/top/') # Getting page HTML through request
# soup = BeautifulSoup(driver.page_source, 'html.parser') # Parsing content using beautifulsoup. Notice driver.page_source instead of page.content
 
# links = soup.select("table tbody tr td.titleColumn a") # Selecting all of the anchors with titles
# first10 = links[:10] # Keep only the first 10 anchors
# for anchor in first10:
#     print(anchor.text) # Display the innerText of each anchor

<ipython-input-4-d9f4ca13aeb4>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", # https://stackoverflow.com/questions/60686871/colab-cant-find-chromedriver-path


In [5]:
driver.get("https://www.banguat.gob.gt/es/page/sistema-de-cuentas-nacionales-trimestrales-ano-de-referencia-2013")
soup = BeautifulSoup(driver.page_source, 'html.parser') # Parsing content using beautifulsoup. Notice driver.page_source instead of page.content

In [6]:
# https://pythonexamples.org/python-selenium-find-element-by-xpath/
# https://www.w3schools.com/xml/xml_xpath.asp
banguat_gdp_link = driver.find_element(By.XPATH, "//table/tbody/tr[2]/td/p[3]/a")#.screenshot("test.png")
banguat_gdp_link

<selenium.webdriver.remote.webelement.WebElement (session="3428194112d8e09a4304ad3eb61dae66", element="00e738ed-f18f-4387-935d-cb6516f18781")>

In [7]:
banguat_gdp_href = banguat_gdp_link.get_attribute("href")
banguat_gdp_href

'https://www.banguat.gob.gt/sites/default/files/banguat/cuentasnac/PIB2013/CNT_cuadros_al_3T_2022.xlsx'

In [8]:
# https://www.codingem.com/python-download-file-from-url/

import requests
URL = banguat_gdp_href
response = requests.get(URL)
file_name = "PIB-Guatemala-sistema-de-cuentas-nacionales-trimestrales-ano-de-referencia-2013-CNT_cuadros.xlsx"
open(file_name, "wb").write(response.content)

136848

In [9]:
# https://colab.research.google.com/notebooks/snippets/gcs.ipynb
from google.colab import auth
auth.authenticate_user()

project_id = 'ufm-market-trends'
!gcloud config set project {project_id}

Updated property [core/project].


In [10]:
import datetime
today = datetime.date.today().strftime("%Y-%m-%d")

In [11]:
# https://stackoverflow.com/questions/18727347/how-to-extract-a-filename-from-a-url-and-append-a-word-to-it
import os
from urllib.parse import urlparse
url = banguat_gdp_href
a = urlparse(url)
print(a.path)                   
original_file_name = os.path.basename(a.path)
# print(os.path.basename(a.path)) 
print(original_file_name)

/sites/default/files/banguat/cuentasnac/PIB2013/CNT_cuadros_al_3T_2022.xlsx
CNT_cuadros_al_3T_2022.xlsx


In [ ]:
bucket_path = "ufm-market-trends/guatemala/web-scrapping/pib-banguat"
xlsx_file = file_name

!gsutil -h "Cache-Control:public, max-age=1" cp {xlsx_file} gs://{bucket_path}
# !gsutil cp gs://{bucket_path}/{xlsx_file} gs://{bucket_path}/{'archivo'}/{today+'-'+xlsx_file}
# Provides more detail to save an archive with the original file name
!gsutil cp gs://{bucket_path}/{xlsx_file} gs://{bucket_path}/{'archivo'}/{today+'-'+original_file_name}
!gsutil cat gs://{bucket_path}/{xlsx_file}

Copying file://PIB-Guatemala-sistema-de-cuentas-nacionales-trimestrales-ano-de-referencia-2013-CNT_cuadros.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][133.6 KiB/133.6 KiB]                                                
Operation completed over 1 objects/133.6 KiB.                                    
Copying gs://ufm-market-trends/guatemala/web-scrapping/pib-banguat/PIB-Guatemala-sistema-de-cuentas-nacionales-trimestrales-ano-de-referencia-2013-CNT_cuadros.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][133.6 KiB/133.6 KiB]                                                
Operation completed over 1 objects/133.6 KiB.                                    
